<a href="https://colab.research.google.com/github/kim1987/aiffel/blob/main/aiffel/exploration/ex_fifteen/ex_fifteen_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

exploration_15  
with pytorch transformer
and https://tutorials.pytorch.kr/beginner/translation_transformer.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import tensorflow as tf

In [2]:
import copy
from typing import Optional, Any, Union, Callable, Tuple

from torch import Tensor
from torch.nn import functional as F
#from torch.nn.module.module import Module
#from torch.nn.module.activation import MultiheadAttention
from torch.nn.modules.container import ModuleList
from torch.nn.init import xavier_uniform_
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.linear import Linear
from torch.nn.modules.normalization import LayerNorm
from torch.nn.parameter import Parameter

In [3]:
from torch.nn import Transformer

In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

데이터 로드 및 전처리

In [5]:
import os
import requests
import pandas as pd
import numpy as np
import re
import csv
down_url = 'https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv'
r = requests.get(down_url)


In [6]:
open('/content/data.csv', 'wb').write(r.content)

3609128

In [7]:
data = pd.read_html('/content/data.csv')

In [8]:
data_change = data[0][1].apply(lambda x : np.array(x.split(',')))

In [9]:
data_test = pd.DataFrame([[data_change[i][0],data_change[i][1],data_change[i][2]] for i in range(1,data_change.count())],columns=data_change[0])

In [10]:
data_frame = pd.DataFrame([[data_change.values[1:][i][j] for j in range(0,3)] for i in range(0,data_change.count()-1)],columns=data_change.values[0])

In [11]:
src_base = data_frame['Q']
tgt_base = data_frame['A']

In [12]:
type(src_base)

pandas.core.series.Series

In [13]:
def preprocess_sentence(sentence):

    sentence = re.sub(r"([?.!,])", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    sentence = sentence.strip()
    return sentence

In [14]:
src_prepro = src_base.apply(preprocess_sentence)
tgt_prepro = tgt_base.apply(preprocess_sentence)

In [15]:
for sen in  src_prepro:
  print(sen)
  break
  

12시 땡


In [16]:
import tensorflow_datasets as tfds

In [17]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(src_prepro + tgt_prepro, target_vocab_size=2**13)

In [18]:
START_TOKEN, END_TOKEN = tokenizer.vocab_size, tokenizer.vocab_size + 1

In [19]:
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

END_TOKEN의 번호 : [8476]


In [20]:
VOCAB_SIZE = tokenizer.vocab_size + 2

In [21]:
MAX_LENGTH = 40

In [22]:
'''
from torch.nn.utils.rnn import pad_sequence
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  for (sentence1, sentence2) in zip(inputs, outputs):
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    mask_1 = np.concatenate([np.zeros(len(sentence1),dtype=np.bool),np.ones(MAX_LENGTH-len(sentence1),dtype=np.bool)])
    mask_2 = np.concatenate([np.zeros(len(sentence2),dtype=np.bool),np.ones(MAX_LENGTH-len(sentence2),dtype=np.bool)])

    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  

  return tokenized_inputs, tokenized_outputs
  '''

"\nfrom torch.nn.utils.rnn import pad_sequence\ndef tokenize_and_filter(inputs, outputs):\n  tokenized_inputs, tokenized_outputs = [], []\n  for (sentence1, sentence2) in zip(inputs, outputs):\n    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN\n    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN\n\n    mask_1 = np.concatenate([np.zeros(len(sentence1),dtype=np.bool),np.ones(MAX_LENGTH-len(sentence1),dtype=np.bool)])\n    mask_2 = np.concatenate([np.zeros(len(sentence2),dtype=np.bool),np.ones(MAX_LENGTH-len(sentence2),dtype=np.bool)])\n\n    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:\n      tokenized_inputs.append(sentence1)\n      tokenized_outputs.append(sentence2)\n  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(\n      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')\n  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(\n      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')\n  \n

In [33]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

In [23]:

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, data):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = data


    def __len__(self):
        return len(self.data[0])

    def __getitem__(self, idx):
        return self.data[0][idx],self.data[1][idx]


In [24]:
my_data = MyDataset((src_prepro,tgt_prepro))

In [25]:
#len(my_data)

In [26]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [34]:
from torch.nn.utils.rnn import pad_sequence

def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids):
    return torch.cat((torch.tensor([START_TOKEN]),
                      torch.tensor(token_ids),
                      torch.tensor([END_TOKEN])))

text_transform = {}
text_transform = sequential_transforms(tokenizer.encode,
                                       tensor_transform)



def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform(src_sample.rstrip("\n")))
        tgt_batch.append(text_transform(tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

make model

In [29]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

파라미터 설정 및 모델 train

In [30]:
 def customize_LR(epoch,d_model,warmup_steps):
    epoch, warmup_steps = torch.Tensor([epoch]),torch.Tensor([warmup_steps])
    arg1 = torch.rsqrt(epoch)
    arg2 = epoch * (warmup_steps**-1.5)
    return d_model * torch.minimum(arg1, arg2).item()

In [31]:
type(torch.Tensor([1]).item())

float

In [52]:
from torch.utils.data import DataLoader
SRC_VOCAB_SIZE = VOCAB_SIZE
TGT_VOCAB_SIZE = VOCAB_SIZE
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
lambda1 = lambda epoch: customize_LR(epoch,d_model=512,warmup_steps=4000)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [37]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_dataloader = DataLoader(my_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)


def evaluate(model):
    model.eval()
    losses = 0

    val_dataloader = DataLoader(my_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

In [53]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    scheduler.step()
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 7.421, Val loss: 6.825, Epoch time = 3.898s
Epoch: 2, Train loss: 6.661, Val loss: 6.415, Epoch time = 3.834s
Epoch: 3, Train loss: 6.350, Val loss: 6.201, Epoch time = 3.845s
Epoch: 4, Train loss: 6.139, Val loss: 6.037, Epoch time = 3.824s
Epoch: 5, Train loss: 5.973, Val loss: 5.887, Epoch time = 3.801s
Epoch: 6, Train loss: 5.830, Val loss: 5.763, Epoch time = 3.829s
Epoch: 7, Train loss: 5.689, Val loss: 5.614, Epoch time = 3.858s
Epoch: 8, Train loss: 5.568, Val loss: 5.494, Epoch time = 3.853s
Epoch: 9, Train loss: 5.463, Val loss: 5.373, Epoch time = 3.807s
Epoch: 10, Train loss: 5.373, Val loss: 5.272, Epoch time = 3.855s
Epoch: 11, Train loss: 5.290, Val loss: 5.199, Epoch time = 3.847s
Epoch: 12, Train loss: 5.221, Val loss: 5.097, Epoch time = 3.862s
Epoch: 13, Train loss: 5.160, Val loss: 5.015, Epoch time = 3.897s
Epoch: 14, Train loss: 5.098, Val loss: 4.941, Epoch time = 3.876s
Epoch: 15, Train loss: 5.045, Val loss: 4.891, Epoch time = 3.882s
Epoc

In [103]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == END_TOKEN:
            break
    return ys


def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 10, start_symbol=START_TOKEN).flatten()
    predicted_sentence = tokenizer.decode([i for i in tgt_tokens if i < tokenizer.vocab_size])
    return predicted_sentence

In [98]:
translate(transformer,'이 정도면 만족')

tensor([123], device='cuda:0')
tensor([3830], device='cuda:0')
tensor([642], device='cuda:0')
tensor([8476], device='cuda:0')


'저도 해보고 싶네요'

In [57]:
translate(transformer,'테스트')

'저도 잘 될 수 있어요'

In [58]:
translate(transformer,'배고파')

'많이 드세요'

In [59]:
translate(transformer,'힘들다')

'잘 될 수 있을 수도 있어요'

In [67]:
translate(transformer,'시간초과했네')

'이제 좀 더 돼요'

In [69]:
translate(transformer,'좋은 사람 만날 수 있을 수도 있어요')

'저도 좀 더 상황을 지켜보세요'

In [72]:
translate(transformer,'같은 문장이 생성된다 그리디서치를 사용해서 그런 걸 수도 있다.')

'좋은 사람 만날 수 있을 수도 있어요'

In [107]:
translate(transformer,'이제 좀 더 돼요')

'많이 지쳤나봐요'

In [108]:
translate(transformer,'많이 지쳤나봐요')

'그 사람도 좀 더 많이 드세요'